In [11]:
import pandas as pd
import pandasql as psql
from geopy.geocoders import Nominatim
import time
import numpy as np

In [12]:
df = pd.read_excel("./data/Cataracts_2325_patient_level.xlsx", engine="openpyxl")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df['independent_sector'] = df['independent_sector'].str.strip().str.title()

print(df.columns)

Index(['activity_type', 'provider', 'local_patient_identifier', 'lsoa', 'hrg',
       'hrg_description', 'fin_year', 'fin_month', 'activity_date',
       'independent_sector'],
      dtype='object')


In [13]:
df['activity_date'] = pd.to_datetime(df['activity_date'], errors='coerce')
df['independent_sector'] = df['independent_sector'].map({'Yes': True, 'No': False})
df['provider_type'] = df['independent_sector'].map({True: 'Independent', False : 'NHS'})

def classify_hrg_detail(hrg_code):
    hrg_map = {
        'BZ30A': ('Complex', 'CC2+', 'High'),
        'BZ30B': ('Complex', 'CC0-1', 'Medium'),
        'BZ31A': ('Very Major', 'CC2+', 'High'),
        'BZ31B': ('Very Major', 'CC0-1', 'Medium'),
        'BZ32A': ('Intermediate', 'CC2+', 'Medium'),
        'BZ32B': ('Intermediate', 'CC0-1', 'Medium'),
        'BZ33Z': ('Minor', 'NA', 'Low'),
        'BZ34A': ('Phaco', 'CC4+', 'High'),
        'BZ34B': ('Phaco', 'CC2-3', 'Medium'),
        'BZ34C': ('Phaco', 'CC0-1', 'Low')
    }
    return hrg_map.get(hrg_code, ('Unknown', 'Unknown', 'Unknown'))

df[['hrg_type', 'cc_score', 'complexity']] = df['hrg'].apply(
    lambda x: pd.Series(classify_hrg_detail(x))
)

df['local_patient_identifier'] = df['local_patient_identifier'].fillna(
    df.index.astype(str) + "_" + df['activity_date'].astype(str)
)

# df[['fin_year_start', 'fin_year_end']] = df['fin_year'].str.split('/', expand=True).astype(int)


df_cleaned = df.drop_duplicates(
    subset=["activity_date", "local_patient_identifier"], keep="first"
)

df_cleaned["needs_ga"] = np.where(
    (df_cleaned["complexity"] == "High") & (np.random.rand(len(df_cleaned)) < 0.3),
    True,
    False
)

print(df_cleaned.isnull().sum())

df_cleaned.to_csv('./data/Cataracts_2325_patient_level_cleaned.csv')

activity_type                  0
provider                       0
local_patient_identifier       0
lsoa                        6124
hrg                            0
hrg_description                0
fin_year                       0
fin_month                      0
activity_date                  0
independent_sector             0
provider_type                  0
hrg_type                       0
cc_score                       0
complexity                     0
needs_ga                       0
dtype: int64


/var/folders/p_/9q2bvzqj04g1v63p4rjk_cxr0000gn/T/ipykernel_51427/2946627899.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["needs_ga"] = np.where(


In [14]:
df_provider_info = pd.read_csv("./data/National_cataract_audit_year_3_data.csv")

df_provider_info.columns

def clean_name(x):
    if isinstance(x, str):
        return x.strip().lower().replace('–', '-').replace('—', '-')
    return x

df_cleaned['provider_clean'] = df_cleaned['provider'].apply(clean_name)
df_provider_info['TrustName_clean'] = df_provider_info['TrustName'].apply(clean_name)


/var/folders/p_/9q2bvzqj04g1v63p4rjk_cxr0000gn/T/ipykernel_51427/1442926436.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['provider_clean'] = df_cleaned['provider'].apply(clean_name)


In [15]:
df_provider_info['TrustName_clean'].value_counts().head(10)

TrustName_clean
care uk                                                  9
spamedica                                                8
portsmouth hospitals nhs trust                           1
bolton nhs foundation trust                              1
james paget university hospitals nhs foundation trust    1
northampton general hospital nhs trust                   1
united lincolnshire hospitals nhs trust                  1
county durham and darlington nhs foundation trust        1
east suffolk and north essex nhs foundation trust        1
east kent hospitals university nhs foundation trust      1
Name: count, dtype: int64

- จำนวนผู้ป่วยทั้งหมด / รายปี / รายเดือน 
- สัดส่วนผู้ป่วยที่ไป Independent vs NHS
- ความซับซ้อนของเคสที่แต่ละ Provider ได้รับ
- อัตราส่วน Provider → NHS vs Independent (Top 5)

In [16]:
# number of patient group by year and month
q1 = """
select  strftime('%Y', activity_date) as year,
        strftime('%m', activity_date) as month,
        count(*) as total_patients 
from df_cleaned
group by year, month
order by year, month
"""
r1 = psql.sqldf(q1, locals())

# q2 = """
# select activity_date, local_patient_identifier, hrg_description, count(*) as count
# from df_cleaned
# group by activity_date, local_patient_identifier, hrg_description
# having count > 1
# """

# patient ratio Independent vs NHS
q2 = """
select 
    (select count(*) 
     from df_cleaned 
     where provider_type = 'Independent') * 1.0 /
    (select count(*)
     from df_cleaned 
     where provider_type = 'NHS') as ind_vs_nhs_ratio
"""
r2 = psql.sqldf(q2, locals())

# total patient od indeoendent and NHS sector
q3 = """
select 
    (select count(*)
     from df_cleaned
     where provider_type = 'Independent') as total_indendent, 
    (select count(*)
     from df_cleaned
     where provider_type = 'NHS') as total_nhs
"""
r3 = psql.sqldf(q3, locals())

# number of patient group by hrg_description and provider type
q4 = """
select strftime('%Y', activity_date) as year,
       provider_type, 
       hrg_type, complexity, 
       hrg_description, 
       count(*) as total
from df_cleaned
group by hrg_description, provider_type, year
order by provider_type, year
"""
r4 = psql.sqldf(q4, locals())

# number of patient group by provider 
q5 = """
select provider, provider_type,count(*) as total
from df_cleaned
group by provider
order by provider_type, total desc
"""
r5 = psql.sqldf(q5, locals())

# number of patient group by provider, hrg_description, fin_month, fin_year
q6 = """
select provider, hrg, hrg_description, fin_year, fin_month, count(*)
from df_cleaned
group by provider, hrg_description, fin_month, fin_year
order by provider
"""
r6 = psql.sqldf(q6, locals())

q7 = """
with provider_trust as (
     select TrustName, 
          ITCLocation, 
          ODSCode, 
          round(NumberHESOperations/3.0, 1) as SlotsPerYear, 
          round(NumberHESOperations/156.0, 1) as SlotsPerWeek,
          round(NumberHESOperations/1092.0, 1) as SlotsPerDay,
          NumberOfSurgeons
          from df_provider_info
),
provider_mapping as (
     select distinct 
          c.provider,
          case when c.independent_sector = 1 then 'true'
               when c.independent_sector = 0 then 'false'
               else 'NA'
          end as is_nhs,
          p.*
          from df_cleaned as c
          left join provider_trust as p
               on lower(c.provider) like lower(p.TrustName || '%')
),
provider_mapping_trust as (
     select distinct
          m.provider,
          m.ODSCode,
          m.TrustName,
          m.is_nhs,
          p.SlotsPerYear, 
          p.SlotsPerWeek,
          p.SlotsPerDay,
          p.NumberOfSurgeons,
          p.ITCLocation,
          case 
               when lower(m.provider) = lower(p.TrustName)
               then 1
               else 0
          end as flag_1,
          case 
               when m.provider LIKE '%' || p.ITCLocation || '%' 
                    AND m.ODSCode = p.ODSCode
               then 1
               else 0
          end as flag_2
     from provider_mapping as m
     left join provider_trust as p
          on m.ODSCode = p.ODSCode
)
select distinct
     provider,
     is_nhs,
     ODSCode,
     TrustName,
     ITCLocation,
     SlotsPerYear,
     SlotsPerWeek,
     SlotsPerDay,
     NumberOfSurgeons
from provider_mapping_trust
where flag_2 = 1

union

select distinct
     provider,
     is_nhs,
     ODSCode,
     TrustName,
     null as ITCLocation,
     case when flag_1 = 1 then SlotsPerYear else null end as SlotsPerYear,
     case when flag_1 = 1 then SlotsPerWeek else null end as SlotsPerWeek,
     case when flag_1 = 1 then SlotsPerDay else null end as SlotsPerDay,
     case when flag_1 = 1 then NumberOfSurgeons else null end as NumberOfSurgeons
from provider_mapping_trust
where provider NOT IN (
     select provider 
     from provider_mapping_trust 
     where flag_2 = 1
     )
"""
r7 = psql.sqldf(q7, locals())
r7.to_csv('./data/provider.csv')


In [17]:
df_provider = pd.read_csv('./data/provider.csv')

In [18]:
q8 = """
with map_slot as (
       select provider,
              round(count(*) * 1.0 / 156.0, 1) as SlotsPerWeek,
              round(count(*) * 1.0 / 3.0, 1) as SlotsPerYear,
              round(count(*) * 1.0 / 1092.0, 1) as SlotsPerDay
       from df_cleaned
       group by provider
)
select p.provider,
       case when p.is_nhs = 1 then 'true'
               when p.is_nhs = 0 then 'false'
               else 'NA'
       end as is_nhs,
       COALESCE(p.SlotsPerYear, m.SlotsPerYear) AS SlotsPerYear,
       COALESCE(p.SlotsPerWeek, m.SlotsPerWeek) AS SlotsPerWeek,
       COALESCE(p.SlotsPerDay, m.SlotsPerDay) AS SlotsPerDay
from df_provider as p
left join map_slot as m
       on p.provider = m.provider

"""
r8 = psql.sqldf(q8, locals())
r8.to_csv('./data/provider2.csv')


In [38]:
df_provider_final = pd.read_csv('./data/provider2.csv')

geolocator = Nominatim(user_agent="ptl-phase1-model")

def geocode_provider(name):
    try:
        location = geolocator.geocode(f"{name}, United Kingdom", timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

df_provider_final[['lat', 'long']] = df_provider_final['provider'].apply(
    lambda x: pd.Series(geocode_provider(x))
)


In [32]:
# df_provider_final.to_csv('./data/provider_lat_long.csv')

In [37]:
q9 = """
select  provider,
        case when independent_sector = 1 then 'true'
             when independent_sector = 0 then 'false'
             else 'NA'
        end as is_nhs,
        strftime('%Y-%W', activity_date) AS year_week,
        count(*) AS all_throughput,
        sum(case when complexity = 'Low' then 1 else 0 end) AS low_complexity,
        sum(case when complexity = 'Medium' then 1 else 0 end) AS medium_complexity,
        sum(case when complexity = 'High' then 1 else 0 end) AS high_complexity
from df_cleaned
group by year_week, provider
order by year_week, provider
"""
r9 = psql.sqldf(q9, locals())
r9.to_csv("./data/provider_weekly_throughput.csv")

In [56]:
df_provider_lat_long_final = pd.read_csv('./data/provider_lat_long.csv')

df_provider_lat_long_final["SlotsPerDay"] = pd.to_numeric(df_provider_lat_long_final["SlotsPerDay"], errors="coerce")

df_provider_lat_long_final["SlotsPerDay"] = df_provider_lat_long_final["SlotsPerDay"].fillna(0)

df_provider_lat_long_final["SlotsPerDay"] = df_provider_lat_long_final["SlotsPerDay"].apply(lambda x: max(1, round(x, 1)))

# df_provider_lat_long_final.to_csv('./data/provider_lat_long.csv')

In [27]:
from __future__ import annotations
import os
from pathlib import Path
from typing import List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _ensure_cols(df: pd.DataFrame, cols: List[str]):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"DataFrame is missing required columns: {missing}")

def _pick_policies(df: pd.DataFrame, wanted: Optional[List[str]]) -> pd.DataFrame:
    if not wanted:
        return df.copy()
    return df[df["policy"].isin(wanted)].copy()

def _safe_round(x, nd=2):
    try:
        return round(float(x), nd)
    except Exception:
        return x

def _compute_overall_share_to_nhs(row: pd.Series) -> float:
    """
    ถ้าไม่มีคอลัมน์ overall_pct_to_nhs จะคำนวณแบบ approx:
    ค่าเฉลี่ยของ {pct_low_to_nhs, pct_medium_to_nhs, pct_high_to_nhs}
    (ถ้ามีสัดส่วนจำนวนผู้ป่วยต่อ complexity สามารถเปลี่ยนเป็น weighted mean ภายหลังได้)
    """
    keys = ["pct_low_to_nhs", "pct_medium_to_nhs", "pct_high_to_nhs"]
    vals = [row[k] for k in keys if k in row and pd.notna(row[k])]
    return float(np.mean(vals)) if vals else np.nan


def fig_avg_wait_by_policy(df: pd.DataFrame, outpath: Path):
    _ensure_cols(df, ["policy", "avg_wait"])
    d = df.copy().sort_values("avg_wait", ascending=True)

    plt.figure(figsize=(9, 5))
    plt.bar(d["policy"], d["avg_wait"])
    plt.xticks(rotation=30, ha="right")
    plt.ylabel("Average wait (days)")
    for i, v in enumerate(d["avg_wait"]):
        plt.text(i, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()


def fig_complexity_mix_stacked(df: pd.DataFrame, outpath: Path):
    needed = [
        "policy",
        "pct_low_to_nhs", "pct_low_to_private",
        "pct_medium_to_nhs", "pct_medium_to_private",
        "pct_high_to_nhs", "pct_high_to_private",
    ]
    _ensure_cols(df, needed)
    d = df.copy()

    segments = [
        ("pct_low_to_nhs", "Low→NHS"),
        ("pct_low_to_private", "Low→Private"),
        ("pct_medium_to_nhs", "Med→NHS"),
        ("pct_medium_to_private", "Med→Private"),
        ("pct_high_to_nhs", "High→NHS"),
        ("pct_high_to_private", "High→Private"),
    ]

    plt.figure(figsize=(11, 6))
    bottoms = [0.0] * len(d)
    for col, label in segments:
        vals = d[col].values
        plt.bar(d["policy"], vals, bottom=bottoms, label=label)
        bottoms = [b + v for b, v in zip(bottoms, vals)]

    plt.xticks(rotation=30, ha="right")
    plt.ylabel("Share of cases (%)")
    plt.legend(loc="upper right", ncol=2, fontsize=9)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()

def fig_tradeoff_scatter(df: pd.DataFrame, outpath: Path):
    _ensure_cols(df, ["policy", "avg_travel_distance_km", "avg_wait"])
    d = df.copy()

    plt.figure(figsize=(7.5, 6))
    plt.scatter(d["avg_travel_distance_km"], d["avg_wait"])
    for _, row in d.iterrows():
        plt.annotate(row["policy"],
                     (row["avg_travel_distance_km"], row["avg_wait"]),
                     xytext=(3, 3), textcoords="offset points", fontsize=9)
    plt.xlabel("Avg travel distance (km)")
    plt.ylabel("Avg wait (days)")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()

def fig_wait_by_complexity(df: pd.DataFrame, outpath: Path):
    _ensure_cols(df, ["policy", "avg_wait_low", "avg_wait_medium", "avg_wait_high"])
    d = df.copy()
    d = d.sort_values("avg_wait", ascending=True) if "avg_wait" in d.columns else d

    labels = d["policy"].tolist()
    low = d["avg_wait_low"].tolist()
    med = d["avg_wait_medium"].tolist()
    high = d["avg_wait_high"].tolist()

    x = np.arange(len(labels))
    width = 0.25

    plt.figure(figsize=(11, 6))
    plt.bar(x - width, low, width, label="Low")
    plt.bar(x,        med, width, label="Medium")
    plt.bar(x + width, high, width, label="High")
    plt.xticks(x, labels, rotation=30, ha="right")
    plt.ylabel("Average wait (days)")
    plt.legend()
    for xi, v in zip(x - width, low):
        plt.text(xi, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=8)
    for xi, v in zip(x, med):
        plt.text(xi, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=8)
    for xi, v in zip(x + width, high):
        plt.text(xi, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=8)

    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()

def plot_complexity_mix(df_summary: pd.DataFrame, policies_order=None, figsize=(12, 6), savepath=None):
    need = [
        "policy",
        "pct_low_to_nhs","pct_low_to_private",
        "pct_medium_to_nhs","pct_medium_to_private",
        "pct_high_to_nhs","pct_high_to_private",
    ]
    _ensure_cols(df_summary, need)

    df = df_summary.copy()
    if policies_order is None:
        policies_order = df["policy"].tolist()
    df = df.set_index("policy").loc[policies_order].reset_index()

    x = np.arange(len(df))
    width = 0.25

    fig, ax = plt.subplots(figsize=figsize)

    # Low
    low_nhs = df["pct_low_to_nhs"].values
    low_pri = df["pct_low_to_private"].values
    ax.bar(x - width, low_nhs, width, label="Low → NHS")
    ax.bar(x - width, low_pri, width, bottom=low_nhs, label="Low → Private")

    # Medium
    med_nhs = df["pct_medium_to_nhs"].values
    med_pri = df["pct_medium_to_private"].values
    ax.bar(x, med_nhs, width, label="Med → NHS")
    ax.bar(x, med_pri, width, bottom=med_nhs, label="Med → Private")

    # High
    hi_nhs = df["pct_high_to_nhs"].values
    hi_pri = df["pct_high_to_private"].values
    ax.bar(x + width, hi_nhs, width, label="High → NHS")
    ax.bar(x + width, hi_pri, width, bottom=hi_nhs, label="High → Private")

    ax.set_xticks(x)
    ax.set_xticklabels(df["policy"], rotation=20, ha="right")
    ax.set_ylabel("Complexity distribution share (%)")
    ax.set_ylim(0, 120)  # กัน rounding overflow

    ax.legend(ncol=3, fontsize=9, frameon=False, loc="upper left", bbox_to_anchor=(0, 1.15))

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=200, bbox_inches="tight")
    plt.close()
    return fig, ax


def _ensure_cols(df: pd.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

def plot_avg_tariff_by_sector(
    df_summary: pd.DataFrame,
    policies_order=None,
    figsize=(12, 4.5),
    savepath=None,
    show_gap_labels=True,
):
    """
    วาดกราฟเปรียบเทียบ 'Avg tariff (NHS)' และ 'Avg tariff (Private)' ต่อ policy
    คาดว่า df_summary มีคอลัมน์: policy, avg_tariff_nhs, avg_tariff_private
    """
    _ensure_cols(df_summary, ["policy", "avg_tariff_nhs", "avg_tariff_private"])
    df = df_summary.copy()

    if policies_order is None:
        policies_order = df["policy"].tolist()
    df = df.set_index("policy").loc[policies_order].reset_index()

    x = np.arange(len(df))
    w = 0.38

    fig, ax = plt.subplots(figsize=figsize)
    b1 = ax.bar(x - w/2, df["avg_tariff_nhs"], width=w, label="NHS")
    b2 = ax.bar(x + w/2, df["avg_tariff_private"], width=w, label="Independent")

    ax.set_ylabel("Average tariff per case (£)")
    ax.set_xticks(x)
    ax.set_xticklabels(df["policy"], rotation=20, ha="right")
    ax.legend()

    # ใส่ตัวเลขบนแท่ง
    for bars in (b1, b2):
        for rect in bars:
            h = rect.get_height()
            ax.text(
                rect.get_x() + rect.get_width()/2, h,
                f"{h:.0f}", ha="center", va="bottom", fontsize=9
            )

    if show_gap_labels:
        gaps = df["avg_tariff_private"] - df["avg_tariff_nhs"]
        for i, g in enumerate(gaps):
            ax.text(
                x[i], max(df["avg_tariff_nhs"][i], df["avg_tariff_private"][i]) + 12,
                f"Δ£{g:.0f}", ha="center", va="bottom", fontsize=9, color="gray"
            )

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=200, bbox_inches="tight")
        plt.close()
    return fig, ax


def tariff_gap_table(df_summary: pd.DataFrame, policies_order=None) -> pd.DataFrame:
    """ทำตารางสรุปตัวเลขไว้ใส่ในรายงาน/ภาคผนวก"""
    _ensure_cols(df_summary, ["policy", "avg_tariff_nhs", "avg_tariff_private"])
    df = df_summary.copy()
    if policies_order is not None:
        df = df.set_index("policy").loc[policies_order].reset_index()
    out = df[["policy", "avg_tariff_nhs", "avg_tariff_private"]].copy()
    out["gap_private_minus_nhs"] = out["avg_tariff_private"] - out["avg_tariff_nhs"]
    # ปัดให้สวย
    return out.rename(columns={
        "avg_tariff_nhs": "Avg tariff NHS (£)",
        "avg_tariff_private": "Avg tariff Independent (£)",
        "gap_private_minus_nhs": "Δ (Ind − NHS) (£)"
    })

def plot_wait_vs_share_nhs(df_summary: pd.DataFrame, policies_order=None, figsize=(10, 6), savepath=None):
    _ensure_cols(df_summary, ["policy", "avg_wait",
                              "pct_low_to_nhs", "pct_medium_to_nhs", "pct_high_to_nhs"])

    df = df_summary.copy()
    if "overall_pct_to_nhs" not in df.columns:
        df["overall_pct_to_nhs"] = df.apply(_compute_overall_share_to_nhs, axis=1)

    if policies_order is None:
        policies_order = df["policy"].tolist()
    df = df.set_index("policy").loc[policies_order].reset_index()

    x = df["overall_pct_to_nhs"].values
    y = df["avg_wait"].values

    fig, ax = plt.subplots(figsize=figsize)
    ax.scatter(x, y)
    for xi, yi, name in zip(x, y, df["policy"]):
        ax.annotate(name, (xi, yi), textcoords="offset points", xytext=(5, 4), fontsize=9)

    ax.set_xlabel("% allocated to NHS (overall, approx)")
    ax.set_ylabel("Average waiting time (days)")

    if len(x) >= 2 and np.isfinite(x).all() and np.isfinite(y).all():
        m, b = np.polyfit(x, y, 1)
        xx = np.linspace(min(x), max(x), 100)
        ax.plot(xx, m * xx + b, linestyle="--")

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=200, bbox_inches="tight")
    plt.close()
    return fig, ax

def plot_seen_within(df: pd.DataFrame, policies_order=None, figsize=(12, 5), savepath=None):
    need = ["policy", "pct_seen_within_7_days", "pct_seen_within_30_days", "pct_seen_within_90_days"]
    _ensure_cols(df, need)

    df = df.copy()
    if policies_order is None:
        policies_order = df["policy"].tolist()
    df = df.set_index("policy").loc[policies_order].reset_index()

    x = np.arange(len(df))
    width = 0.25

    fig, ax = plt.subplots(figsize=figsize)
    ax.bar(x - width, df["pct_seen_within_7_days"], width, label="Within 7 days")
    ax.bar(x,        df["pct_seen_within_30_days"], width, label="Within 30 days")
    ax.bar(x + width, df["pct_seen_within_90_days"], width, label="Within 90 days")

    ax.set_xticks(x)
    ax.set_xticklabels(df["policy"], rotation=20, ha="right")
    ax.set_ylabel("% of patients seen")
    ax.legend()

    for xi, v in zip(x - width, df["pct_seen_within_7_days"]):
        ax.text(xi, v, f"{v:.1f}", ha="center", va="bottom", fontsize=8)
    for xi, v in zip(x, df["pct_seen_within_30_days"]):
        ax.text(xi, v, f"{v:.1f}", ha="center", va="bottom", fontsize=8)
    for xi, v in zip(x + width, df["pct_seen_within_90_days"]):
        ax.text(xi, v, f"{v:.1f}", ha="center", va="bottom", fontsize=8)

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=200, bbox_inches="tight")
    plt.close()
    return fig, ax


def plot_wait_ga_vs_nonga(df: pd.DataFrame, policies_order=None, figsize=(10, 5), savepath=None):
    need = ["policy", "avg_wait_ga", "avg_wait_non_ga"]
    _ensure_cols(df, need)

    df = df.copy()
    if policies_order is None:
        policies_order = df["policy"].tolist()
    df = df.set_index("policy").loc[policies_order].reset_index()

    x = np.arange(len(df))
    width = 0.35

    fig, ax = plt.subplots(figsize=figsize)
    ax.bar(x - width/2, df["avg_wait_ga"], width, label="GA patients")
    ax.bar(x + width/2, df["avg_wait_non_ga"], width, label="Non-GA patients")

    ax.set_xticks(x)
    ax.set_xticklabels(df["policy"], rotation=20, ha="right")
    ax.set_ylabel("Average wait (days)")
    ax.legend()

    for xi, v in zip(x - width/2, df["avg_wait_ga"]):
        ax.text(xi, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=8)
    for xi, v in zip(x + width/2, df["avg_wait_non_ga"]):
        ax.text(xi, v, f"{_safe_round(v,1)}", ha="center", va="bottom", fontsize=8)

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=200, bbox_inches="tight")
    plt.close()
    return fig, ax

def make_all_figures(
    df: pd.DataFrame,
    outdir: str = "figs",
    policies_keep: Optional[List[str]] = None,
    sort_for_readability: bool = True,
):
    """
    df: summary table (หนึ่งแถวต่อ policy)
    policies_keep: รายชื่อ policy ที่จะโชว์ (จะคงลำดับตามที่ระบุ)
    """
    os.makedirs(outdir, exist_ok=True)
    d = df.copy()

    if policies_keep:
        d = _pick_policies(d, policies_keep)
        d["policy"] = pd.Categorical(d["policy"], categories=policies_keep, ordered=True)
        d = d.sort_values("policy")

    d["policy"] = d["policy"].astype(str).str.strip()

    p1  = Path(outdir) / "nappear/fig1_avg_wait_by_policy_nappear.png"
    p2  = Path(outdir) / "nappear/fig2_complexity_mix_stacked_nappear.png"
    p3  = Path(outdir) / "nappear/fig3_tradeoff_wait_vs_distance_nappear.png"
    p4  = Path(outdir) / "nappear/fig4_wait_by_complexity_nappear.png"
    p5a = Path(outdir) / "nappear/fig5a_complexity_mix_by_group_nappear.png"
    p5b = Path(outdir) / "nappear/fig5b_avg_tariff_by_sector_nappear.png"
    pWT = Path(outdir) / "nappear/fig_wait_vs_nhs_share_nappear.png"
    p6a = Path(outdir) / "nappear/fig6a_seen_within_nappear.png"
    p6b = Path(outdir) / "nappear/fig6b_wait_ga_vs_non_ga_nappear.png"

    fig_avg_wait_by_policy(d, p1)
    fig_complexity_mix_stacked(d, p2)
    fig_tradeoff_scatter(d, p3)
    fig_wait_by_complexity(d, p4)
    plot_complexity_mix(d, policies_order=list(d["policy"]), savepath=p5a)
    # วาด NHS vs Independent tariff (ต้องมีคอลัมน์ avg_tariff_nhs & avg_tariff_private)
    plot_avg_tariff_by_sector(d, policies_order=list(d["policy"]), savepath=p5b)
    plot_wait_vs_share_nhs(d, policies_order=list(d["policy"]), savepath=pWT)
    plot_seen_within(d, policies_order=list(d["policy"]), savepath=p6a)
    plot_wait_ga_vs_nonga(d, policies_order=list(d["policy"]), savepath=p6b)

    print("[Saved]")
    for p in [p1, p2, p3, p4, p5a, p5b, pWT, p6a, p6b]:
        print(p.resolve())


def make_all_figures_from_csv(
    csv_path: str,
    outdir: str = "figs",
    policies_keep: Optional[List[str]] = None,
):
    df = pd.read_csv(csv_path)
    make_all_figures(df, outdir=outdir, policies_keep=policies_keep)


if __name__ == "__main__":
    pass

In [28]:
import pandas as pd

df = pd.read_csv("./data/summary_results/summary_all_policies_old.csv")

keep = [
    # "Baseline",
    "Shared Equal",
    "Fastest First",
    # "Complexity Balanced",
    # "Fee Biased",
    "Neutral Fee",
    # "ML Balanced (wait+dist+util)",
]


make_all_figures(df, outdir="figs", policies_keep=keep)

[Saved]
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig1_avg_wait_by_policy_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig2_complexity_mix_stacked_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig3_tradeoff_wait_vs_distance_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig4_wait_by_complexity_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig5a_complexity_mix_by_group_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig5b_avg_tariff_by_sector_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig_wait_vs_nhs_share_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig6a_seen_within_nappear.png
/Users/nattanan/Desktop/Dissertation/ptl_sim/figs/nappear/fig6b_wait_ga_vs_non_ga_nappear.png


In [31]:
# plot_frontier.py

CSV_PATH = Path("/Users/nattanan/Desktop/Dissertation/ptl_sim/data/summary_results/nhs_share_frontier_ci_by_target.csv")
OUT_DIR = Path("./fig_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(CSV_PATH)

required_cols = [
    "target_high_to_private_share",
    "avg_wait_mean", "avg_wait_ci_half",
    "nhs_share_mean", "nhs_share_ci_half",
    "avg_travel_distance_mean", "avg_travel_distance_ci_half",
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

x = df["target_high_to_private_share"]

def plot_with_ci(x, y_mean, y_ci, xlabel, ylabel, title, outfile):
    y_lo = y_mean - y_ci
    y_hi = y_mean + y_ci

    plt.figure(figsize=(6, 4))
    plt.plot(x, y_mean, marker="o")
    plt.fill_between(x, y_lo, y_hi, alpha=0.2, linewidth=0)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"{outfile}.png", dpi=300)
    plt.savefig(OUT_DIR / f"{outfile}.pdf")
    plt.close()

plot_with_ci(
    x=x,
    y_mean=df["avg_wait_mean"],
    y_ci=df["avg_wait_ci_half"],
    xlabel="Target High→Private share",
    ylabel="Average wait (days)",
    title="Average wait vs Target High→Private share",
    outfile="frontier_avg_wait"
)

plot_with_ci(
    x=x,
    y_mean=df["nhs_share_mean"],
    y_ci=df["nhs_share_ci_half"],
    xlabel="Target High→Private share",
    ylabel="NHS share (%)",
    title="NHS share vs Target High→Private share",
    outfile="frontier_nhs_share"
)

plot_with_ci(
    x=x,
    y_mean=df["avg_travel_distance_mean"],
    y_ci=df["avg_travel_distance_ci_half"],
    xlabel="Target High→Private share",
    ylabel="Average travel distance (km)",
    title="Travel distance vs Target High→Private share",
    outfile="frontier_travel_distance"
)

fig, axes = plt.subplots(3, 1, figsize=(6.2, 8), sharex=True)

def _subplot(ax, x, y_mean, y_ci, ylabel, title):
    y_lo, y_hi = y_mean - y_ci, y_mean + y_ci
    ax.plot(x, y_mean, marker="o")
    ax.fill_between(x, y_lo, y_hi, alpha=0.2, linewidth=0)
    ax.set_ylabel(ylabel)
    ax.set_title(title, pad=6)
    ax.grid(True, alpha=0.3)

_subplot(
    axes[0], x,
    df["avg_wait_mean"], df["avg_wait_ci_half"],
    "Avg wait (days)", "Average wait"
)
_subplot(
    axes[1], x,
    df["nhs_share_mean"], df["nhs_share_ci_half"],
    "NHS share (%)", "NHS share"
)
_subplot(
    axes[2], x,
    df["avg_travel_distance_mean"], df["avg_travel_distance_ci_half"],
    "Travel (km)", "Average travel distance"
)

axes[-1].set_xlabel("Target High→Private share")
plt.tight_layout()
plt.savefig(OUT_DIR / "frontier_combined.png", dpi=300)
plt.savefig(OUT_DIR / "frontier_combined.pdf")
plt.close()

print("Saved figures to:", OUT_DIR.resolve())


Saved figures to: /Users/nattanan/Desktop/Dissertation/ptl_sim/fig_out
